# Proof of Concept

Figures to recreate:
* Coverage vs. Atypicality Quantile - for any CP algorithm, data generation, and atypicality score setting 
* Efficiency vs. Atypicality (cut?)
* Proof of Concept (3x3) - for true atypicality
* Proof of Concept (3x3) - for predicted atypicality
* Distribution of True value in the interval

In [3]:
import sys
sys.path.append("../src")

from atypicality import compute_atypicality_scores
from data_generation_settings import generate_and_split_gaussian_data, generate_and_split_lognormal_data, generate_and_split_gmm_data
from fit_cp_models import fit_rf_cp_model, predict_cp_intervals
from compute_bounds import run_calibration_for_score
from plotting import plot_with_error_bars

In [8]:
atypicality_settings = [
    # ('knn_score', 'aar_knn_lam1_lower', 'aar_knn_lam1_upper', 1),
    # ('kde_score', 'aar_kde_lower', 'aar_kde_upper', 1),
    ('log_joint_mvn_score', 'aar_logjointmvn_lower', 'aar_logjointmvn_upper', 1)
    # ('lognormal_score', 'aar_lognormal_lower', 'aar_lognormal_upper', 1)
]

# Run calibration across all atypicality types
all_results = run_calibration_for_score(atypicality_settings, 
                                        make_and_split_data=generate_and_split_lognormal_data, 
                                        fit_cp_model=fit_rf_cp_model, 
                                        n_samples=500,
                                        n_splits=3, 
                                        test_atypicality=True)

# Plot each atypicality type separately
for atyp_col, coverage_dfs in all_results.items():
    atypicality_score_title = atyp_col.replace('_', ' ').title()
    plot_with_error_bars(coverage_dfs, atypicality_score_title)

TypeError: run_calibration_for_score() got an unexpected keyword argument 'n_samples'

In [7]:
all_results['log_joint_mvn_score_lam1'][0]

,Quantile,Coverage_AAR,Coverage_Pred
0,0,0.095,0.985
1,1,0.525,0.950
2,2,0.880,0.895
3,3,1.000,0.745
4,4,1.000,0.480
